## "For" loops

In [1]:
import numpy as np
import numba

@numba.jit(nopython=True)
def numba_loops(arr):
    n = arr.shape[0]
    result = 0.0
    for i in range(n):
        result += arr[i] ** 2
    return result

In [118]:
%%timeit
arr = np.random.rand(1000000)
numba_loops(arr)  # Very fast

11.2 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [119]:
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
from jax import jit

@jit
def jax_loops(arr):
    n = arr.shape[0]
    result = 0.0
    for i in range(n):  # JAX does not optimize this well
        result += arr[i] ** 2
    return result

In [4]:
%%time
arr = jnp.array(np.random.rand(100))
print(jax_loops(arr))  # Not as fast as Numba

37.300316
CPU times: total: 93.8 ms
Wall time: 184 ms


In [5]:
def python_loops(arr):
    n = arr.shape[0]
    result = 0.0
    for i in range(n):  # JAX does not optimize this well
        result += arr[i] ** 2
    return result

In [6]:
%%time
arr = jnp.array(np.random.rand(10**4))
print(python_loops(arr))  # Not as fast as Numba

3342.7043
CPU times: total: 1.12 s
Wall time: 1.44 s


Very bad results when using Python for and JAX

## ODE solver

#### JAX approach

In [7]:
import jax.numpy as jnp
import jax.lax as lax
from jax import jit

def f(x, t):  
    return -x  # Example: Simple exponential decay dx/dt = -x

def step(carry, t):
    x, h = carry
    x_new = x + h * f(x, t)  # Euler step
    return (x_new, h), x_new  # (carry, output)

In [8]:
@jit
def solve_euler(x0, h, t_array):
    carry = (x0, h)
    carry, x_values = lax.scan(step, carry, t_array)
    return x_values  # Solution for all timesteps

In [9]:
n_steps = 10**5

In [10]:
t_array = jnp.linspace(0, 10, n_steps)  # Time steps
h = t_array[1] - t_array[0]  # Step size
x0 = jnp.array(1.0)  # Initial condition

In [11]:
%%timeit

solution = solve_euler(x0, h, t_array)

157 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


#### Naïve approach

In [12]:
def euler_naive(x0, h, t_array):
    x = np.zeros(len(t_array))
    x[0] = x0
    for i, t in enumerate(t_array[:-1]):
        x[i + 1] = x[i] + h * f(x[i], t)  # Euler update

    return np.array(x)

In [13]:
%%timeit

solution = euler_naive(x0, h, t_array)

3.1 s ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Numba

In [14]:
import numpy as np
import numba

@numba.njit
def solve_euler_numba(x0, h, t_array):
    n = t_array.shape[0]
    x_values = np.empty(n, dtype=np.float64)  
    x_values[0] = x0
    
    for i in range(n - 1):
        x_values[i + 1] = x_values[i] + h * (-x_values[i])  # dx/dt = -x

    return x_values

In [15]:
t_array_np = np.linspace(0, 10, n_steps)  # Time steps
h_np = t_array_np[1] - t_array_np[0]  # Step size
x0_np = np.array(1.0)  # Initial condition

In [16]:
%%timeit
solution = solve_euler_numba(x0_np, h_np, t_array_np)

196 µs ± 4.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Jax gives the best results for many timesteps when used wisely

## Creation of shuffle tables

In [8]:
%load_ext autoreload
%autoreload 2
import sys
if "../.." not in sys.path:
    sys.path.append("../..")

from signature.signature.jax_signature.shuffle_table import get_shuffle_table

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
table_trunc = 10
dim = 2

In [23]:
%%time
table_numba = ShuffleOperator(table_trunc, Alphabet(dim)).shuffle_table.T

CPU times: total: 1.88 s
Wall time: 2.12 s


In [24]:
table_numba

array([[   0,    0,    0,    1],
       [   0,    1,    1,    1],
       [   0,    2,    2,    1],
       ...,
       [2044,    0, 2044,    1],
       [2045,    0, 2045,    1],
       [2046,    0, 2046,    1]], dtype=int64)

In [28]:
%%time
table_jax = get_shuffle_table(table_trunc=table_trunc, dim=dim).T

CPU times: total: 1.81 s
Wall time: 2.18 s


In [29]:
table_jax

array([[   0,    0,    0,    1],
       [   0,    1,    1,    1],
       [   0,    2,    2,    1],
       ...,
       [2044,    0, 2044,    1],
       [2045,    0, 2045,    1],
       [2046,    0, 2046,    1]])

In [30]:
np.allclose(table_jax, table_numba)

True

## Shuffle product (when the shuffle table is compiled)

In [5]:
%load_ext autoreload
%autoreload 2
import sys
if "../.." not in sys.path:
    sys.path.append("../..")

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import json
from signature.signature.tensor_algebra import TensorAlgebra
from signature.signature.tensor_sequence import TensorSequence
from signature.signature.stationary_signature import stationary_signature_from_path, G
from signature.signature.expected_signature import expected_signature, expected_stationary_signature
from scipy.optimize import minimize

green = "#5b6c64"
copper = "#B56246"
plt.rcParams["figure.figsize"]        = ((1 + np.sqrt(5)) / 2 * 5, 5)
plt.rcParams["figure.autolayout"]     = True
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams["axes.grid"]             = True
plt.rcParams['axes.prop_cycle']       = matplotlib.cycler(color=[green, copper, "#322B4D", "#28BC9C", "#71074E"])

ModuleNotFoundError: No module named 'signature.signature.tensor_algebra'

#### Numba

In [32]:
%%time

table_trunc = 10
ts_trunc = 5

ta = TensorAlgebra(dim=2, trunc=table_trunc)

Compiling...
Compilation finished.
CPU times: total: 1.38 s
Wall time: 2.26 s


In [33]:
rng = np.random.default_rng(seed=42)
array = rng.random(size=ta.alphabet.number_of_elements(ts_trunc))

In [34]:
ts = ta.from_array(trunc=table_trunc, array=array)

In [35]:
%%timeit
res = ta.shuop.shuffle_prod(ts, ts)

4.49 ms ± 320 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [36]:
res = ta.shuop.shuffle_prod(ts, ts)
res.array.squeeze().real

array([ 0.59900797,  0.67934525,  1.32903411, ..., 79.59349657,
       67.10270751, 80.51194773])

#### Pure numpy

In [37]:
def __get_extended_array_np(ts: TensorSequence):
    n_elements = ta.alphabet.number_of_elements(table_trunc)

    new_array = np.zeros((n_elements,) + ts.shape[1:], dtype=complex)
    new_array[:min(n_elements, ts.shape[0])] = ts.array[:min(n_elements, ts.shape[0])]
    return new_array

def shuffle_prod_np(
    ts1: TensorSequence,
    ts2: TensorSequence,
    shuffle_table
):
    index_left, index_right, index_result, count = shuffle_table

    if ts1.trunc < table_trunc:
        array_1 = __get_extended_array_np(ts1)
    else:
        array_1 = ts1.array

    if ts2.trunc < table_trunc:
        array_2 = __get_extended_array_np(ts2)
    else:
        array_2 = ts2.array

    source = count * array_1[index_left, 0, 0] * array_2[index_right, 0, 0]
    linear_result = np.zeros(index_result[-1] + 1, dtype=complex)
    for i in range(len(index_result)):
        linear_result[index_result[i]] = linear_result[index_result[i]] + source[i]
    return TensorSequence(ta.alphabet, table_trunc, linear_result)

In [38]:
%%timeit
res2 = shuffle_prod_np(ts, ts, ta.shuop.shuffle_table)

112 ms ± 9.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
res2 = shuffle_prod_np(ts, ts, ta.shuop.shuffle_table)
ta.print(res2 - res)

In [ ]:
import jax
import jax.numpy as jnp
from jax import lax
from functools import partial

from signature.signature.jax_signature.words import number_of_words_up_to_trunc

def change_trunc(ts: jax.Array, trunc: int, dim: int = 2):
    n_elements = number_of_words_up_to_trunc(trunc, dim)
    new_array = jnp.zeros(n_elements)
    new_array = new_array.at[:min(n_elements, ts.shape[0])].set(ts[:min(n_elements, ts.shape[0])])
    return new_array

@jax.jit
def shuffle_prod_jax(
    ts1: jax.Array,
    ts2: jax.Array,
    shuffle_table: jax.Array,
):
    index_left, index_right, index_result, count = shuffle_table

    source = count * ts1[index_left] * ts2[index_right]
    linear_result = ts1 * 0
    #jnp.zeros(number_of_words_up_to_trunc(trunc)) # index_result[-1] + 1
    linear_result = linear_result.at[index_result].add(source)
    
    return linear_result

shuffle_prod_jax_vect = jax.jit(jax.vmap(shuffle_prod_jax, in_axes=(1, 1, None), out_axes=1))

In [ ]:
ts_arr = jnp.array(ts.array.squeeze()).real
ts_arr

Array([0.77395605, 0.43887844, 0.85859792, ..., 0.        , 0.        ,
       0.        ], dtype=float64)

In [ ]:
ts_arr_2d = jnp.vstack([ts_arr] * 10).T

In [ ]:
shuffle_table = get_shuffle_table(table_trunc=table_trunc)

In [ ]:
%%timeit
ts_arr_2 = change_trunc(ts=ts_arr, trunc=10)
ts_arr_2 = change_trunc(ts=ts_arr, trunc=10)
shuffle_prod_jax(ts_arr, ts_arr, shuffle_table=shuffle_table)

4.02 ms ± 239 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [49]:
import jax
import jax.numpy as jnp
from typing import Callable
from signature.signature.jax_signature.tensor_sequence_jax import TensorSequenceJAX
from signature.signature.jax_signature.operators import discount_ts, semi_integrated_scheme

from functools import partial

@partial(jax.jit, static_argnames=['fun'])
def ode_stat_pece(
    fun: Callable,
    t_grid: jax.Array,
    u: TensorSequenceJAX,
    lam: float
) -> TensorSequenceJAX:
    dt = jnp.diff(t_grid)
    
    def step_fun(i, psi):
        psi_pred = discount_ts(ts=psi, dt=dt[i], lam=lam) + semi_integrated_scheme(ts=fun(psi), dt=dt[i], lam=lam)
        psi_next = discount_ts(ts=psi, dt=dt[i], lam=lam) + semi_integrated_scheme(ts=(fun(psi_pred) + fun(psi)) * 0.5, dt=dt[i], lam=lam)
        return psi_next
    
    psi_res = jax.lax.fori_loop(lower=0, upper=len(dt), body_fun=step_fun, init_val=u * 1)
    
    return psi_res

In [50]:
from signature.signature.jax_signature.words import number_of_words_up_to_trunc

In [51]:
t_grid = jnp.linspace(0, 1, 100)
func = lambda ts: ts * (-1)
u = TensorSequenceJAX(jnp.ones(number_of_words_up_to_trunc(5, 2)), 5, 2)
lam = 3

In [52]:
ode_stat_pece(func, t_grid, u, lam)

[3.67885745e-01 1.83131283e-02 1.83131283e-02 9.11335168e-04
 9.11335168e-04 9.11335168e-04 9.11335168e-04 4.53377142e-05
 4.53377142e-05 4.53377142e-05 4.53377142e-05 4.53377142e-05
 4.53377142e-05 4.53377142e-05 4.53377142e-05 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07]

In [37]:
ode_stat_pece(func, t_grid, u, lam)

[3.67885745e-01 1.83131283e-02 1.83131283e-02 9.11335168e-04
 9.11335168e-04 9.11335168e-04 9.11335168e-04 4.53377142e-05
 4.53377142e-05 4.53377142e-05 4.53377142e-05 4.53377142e-05
 4.53377142e-05 4.53377142e-05 4.53377142e-05 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 2.25479375e-06
 2.25479375e-06 2.25479375e-06 2.25479375e-06 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07 1.12103577e-07
 1.12103577e-07 1.12103577e-07 1.12103577e-07]